In [33]:
from sklearn.cluster import KMeans
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
import re
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

import config

In [2]:
final_df = pd.read_csv(config.final_df)
final_df['preprocessed'] = final_df['preprocessed'].apply(lambda x:eval(x))

### 1. Leave only text in brackets

In [4]:
def leave_only_in_brackets(x):
    return re.findall('\[(.*?)\]', x)

In [5]:
final_df = final_df[final_df.text.notna()]

In [6]:
final_df['no_references'] = final_df['text'].progress_apply(lambda x: leave_only_in_brackets(x))

100%|██████████| 63018/63018 [00:00<00:00, 102816.42it/s]


### 2. Find a cluster with words that obviously belong to author's comments

In [7]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20)

sent = final_df['preprocessed'].values 
w2v_model.build_vocab(sent, progress_per=10000)
w2v_model.train(sent, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(78430091, 159846210)

In [8]:
vectors = []
for word in w2v_model.wv.vocab:
    vectors.append(w2v_model[word])

/home/marie/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
kmeans = KMeans(
     init="random",
     n_clusters=15,
     n_init=10,
     max_iter=300,
     random_state=42
 )

In [10]:
clusters = kmeans.fit_predict(vectors)

In [11]:
noisy_df = pd.DataFrame(data={'word': [word for word in w2v_model.wv.vocab], 'cluster':clusters})

In [102]:
noisy = set(noisy_df[noisy_df.cluster == 8].word.values)

In [103]:
def remove_comments(text):
    text = re.sub("[^А-Яа-я']+", ' ', str(text)).lower().split(' ')
    text = [x for x in text if x != '']
    true_text = []
    for word in text:
        word = re.sub("[^А-Яа-я']+", ' ', str(word)).lower()
        if mystem.lemmatize(word.lower())[0] not in noisy:
            true_text.append(word)
    return ' '.join(true_text)

In [110]:
final_df['no_references_cleaned'] = final_df['no_references'].progress_apply(remove_comments)

100%|██████████| 63018/63018 [04:03<00:00, 259.20it/s]


In [108]:
remove_comments(final_df.iloc[258]['text'])

'м султонназаровой г моя бабушка поучала чтобы расчесывая волосы я не выбрасывала бы падаюшие волосы во двор иначе кафиры соберут их и затемнят ими луну в кишлаке манем сельсове сучан р н шугнан растет дерево которую называют видуг саксаул у него узкие длинные красно зеленые листья кафиры затемняют луну ветками из этого дерева чтобы затмение кончилось женщины поднимаются на крышу и с криком бьют в барабаны и бубны лашкарбеков'

In [109]:
final_df.iloc[258]['text']

' [записано М. Султонназаровой, 2008 г.; 1) моя бабушка поучала, чтобы расчесывая волосы, я не выбрасывала бы падаюшие волосы во двор; иначе кафиры соберут их и затемнят ими Луну; 2) в кишлаке Манем (сельсове Сучан, р-н. Шугнан), растет дерево, которую называют Видуг (саксаул); у него узкие длинные красно-зеленые листья; кафиры затемняют Луну ветками из этого дерева; чтобы затмение кончилось, женщины поднимаются на крышу и с криком бьют в барабаны и бубны]: Лашкарбеков, рукопись; '

### 3. Find the most rare words

In [180]:
texts = final_df['no_references'].values

In [181]:
all_words = []
for text in texts[1:]:
    try:
        all_words.append(text[0].split(' '))
    except IndexError:
        continue

In [182]:
all_words = np.concatenate(all_words)

In [184]:
mystem = Mystem() 

In [185]:
lemmatized_words = []
for word in tqdm_notebook(all_words):
    try:
        lemmatized_words.append(mystem.lemmatize(word.lower())[0])
    except IndexError:
        continue

/home/marie/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [187]:
lemmatized_words = pd.DataFrame(data={'words': lemmatized_words})

In [204]:
frequency = pd.DataFrame(lemmatized_words.groupby('words')['words'].value_counts().sort_values().droplevel(1))

In [243]:
rare_words = set(frequency[frequency['words'] < 5].index.values)

In [244]:
def remove_rare_words(text):
    text = re.sub("[^А-Яа-я']+", ' ', str(text)).lower().split(' ')
    text = [x for x in text if x != '']
    true_text = []
    for word in text:
        word = re.sub("[^А-Яа-я']+", ' ', str(word)).lower()
        if mystem.lemmatize(word.lower())[0] not in rare_words:
            true_text.append(word)
    return ' '.join(true_text)

In [245]:
final_df['no_references_cleaned'] = final_df['no_references_cleaned'].progress_apply(remove_rare_words)

100%|██████████| 63018/63018 [03:55<00:00, 267.29it/s]


In [246]:
final_df.iloc[258]['no_references_cleaned']

"'записано м г моя бабушка чтобы расчесывая волосы я не выбрасывала бы волосы во двор иначе соберут их и затемнят ими луну в кишлаке манем сельсове р н растет дерево которую называют саксаул у него узкие длинные красно зеленые листья затемняют луну ветками из этого дерева чтобы затмение кончилось женщины поднимаются на крышу и с криком бьют в барабаны и бубны'"

In [247]:
final_df.to_csv(config.final_df, index=False)